In [16]:
import numpy as np
import pandas as pd
import itertools as itr

In [2]:
# path
transition = "/Users/keiikegami/Dropbox/CREPE_tax/overall/transition.csv"
tax_param = "/Users/keiikegami/Dropbox/CREPE_tax/credits/tax_params.csv"
basic_deduction = "/Users/keiikegami/Dropbox/CREPE_tax/deductions/basic_deduction.csv"
employment_income = "/Users/keiikegami/Dropbox/CREPE_tax/deductions/employment_income.csv"
exemption_for_dependents = "/Users/keiikegami/Dropbox/CREPE_tax/deductions/exemption_for_dependents.csv"
f_window_deduction = "/Users/keiikegami/Dropbox/CREPE_tax/deductions/f_window_deduction.csv"
old_age_deduction = "/Users/keiikegami/Dropbox/CREPE_tax/deductions/old_age_deduction.csv"
pension_deduction = "/Users/keiikegami/Dropbox/CREPE_tax/deductions/pension_deduction.csv"
spouse_deduction = "/Users/keiikegami/Dropbox/CREPE_tax/deductions/spouse_deduction.csv"
spouse_special_deduction = "/Users/keiikegami/Dropbox/CREPE_tax/deductions/spouse_special_deduction.csv"
working_student_deduction = "/Users/keiikegami/Dropbox/CREPE_tax/deductions/working_student_deduction.csv"

# parameter retrieve

In [3]:
# one year
def OneYearParam(year, file_paths, file_names):
    transition_table = pd.read_csv(file_paths[0])
    version = transition_table[transition_table["year"] == year]
    param_set = {}
    
    for (file_path, file_name) in zip(file_paths[1:], file_names[1:]):
        file = pd.read_csv(file_path)
        params = file[file["version"] == version["%s" %file_name].values[0]]
        param_set["%s" %file_name] = params
        
    return param_set

# multiple years
# years must be list
def MultipleYearsParam(years, file_paths, file_names):
    params = {}
    for year in years:
        param[year] = OneYearParam(year, file_paths, file_names)
    return params

In [4]:
file_paths = [transition, basic_deduction, tax_param, employment_income, exemption_for_dependents, f_window_deduction, old_age_deduction, pension_deduction, spouse_deduction, spouse_special_deduction, working_student_deduction]
file_names = ["transition", "basic_deduction", "tax_rates", "employment_income", "exemption_for_dependents", "f_window_deduction", "old_age_deduction", "pension_deduction", "spouse_deduction", "spouse_special_deduction", "working_student_deduction"]
year = 2000

param_2000 = OneYearParam(year, file_paths, file_names)

In [7]:
# このように条件分岐を含むので複数行のテーブルもあることに注意
param_2000["old_age_deduction"]

,version,min_income,max_income,min_age,max_age,fixed_deduction
28,v88,0,1000.0,0,65.0,NaN
29,v88,0,1000.0,65,NaN,50.0
30,v88,1000,NaN,0,65.0,NaN
31,v88,1000,NaN,65,NaN,NaN


# calculation

全体の流れ
1. householdのデータを読み込む
2. timeごとに分割（してアクセス）
3. timeごとにパラメータを呼び出して、全てのpolicy classのインスタンスを作成
4. dummyごとに必要な控除を並列計算
5. 最後に全ての家系に対して同時に控除後所得から所得税を計算


## household

In [10]:
# household tableみたいなのを作成するようにする
# それに対して行列で一気に控除を計算できるようにして高速化

In [11]:
sample = pd.DataFrame(np.array([[2,3],[4,5]]))
sample["dependent_dummy"] = [0,1]
sample["widow_dummy"] = [0,1]
sample["old_dummy"] = [1,1]
sample["pension_dummy"] = [0,1]
sample["spouse_dummy"] = [1,0]
sample["workstu_dummy"] = [0,1]

In [17]:
sample["dependent_dummy"].values

array([0, 1])

## tax policy

In [17]:
# こっちは控除の種類ごとにparameterを入れたら条件分岐が定まるように関数を書く

In [ ]:
# working student
# こいつの引数は子供所得状況で、複数人子供がいればそいつらについてたす
# でもそれをwideに保存するとNANが出すぎるので、
# 子供の数でtableを分割しようかなという感じ→そのために勤労学生の数をhouseholdに与えておく
class workingstudent:
    
    def __init__(self, df_workstu):
        self.df = df_workstu
        self.deduction = self.df["fixed_deduction"]
        self.total_income_threshold = self.df["max_total_income"]
        self.employ_income_threshold = self.df["employment_income"]
        
    def Condition(self, total_inc, emp_inc):
        
        if total_inc < self.total_income_threshold:
            
            if emp_inc < self.employ_income_threshold:
                
                return self.deduction
    
    def CalDeduc(self, household):
        
        for num in np.unique(household["num_workstu"].values):
            df_num = self.df[df["num_workstu"] == num]
            
            for i in range(num):
                df_num["working_student_deduction"] = df_num.apply()


In [9]:
np.unique(np.array([2,3,4,5,3,3,5,3,4,5,4,2]))

array([2, 3, 4, 5])

In [18]:
# データはロングで入ってくる
# 家計idが共通で、子供とか妻とかの収入とかが入っている。
# 構成員全てについて同じ変数が聞かれている
# 納税者ごとに控除がかかるかどうかが決まる（家計ごとではない）
# まず家計を見てその構成員がそれぞれどの控除にかかっているのかを判別する関数が欲しい

# 老年者、配偶者特別、寡婦控除（男女）、配偶者控除、扶養者控除

# 所得と勤労状況を見て誰が誰の扶養になっているかを決める関数
# relationsを見て各納税者がどの控除にひっかかっているかを判断する関数
#この二つがあれば、あとは各控除のダミー変数でクエリして計算させる

# 寡婦控除以外はほぼ全てandの条件なのでNAじゃないのおは各versionに一つしかない
# 寡婦控除はorの条件があるのでその時はNAじゃないのが複数あるため注意

# main part

## household example data

In [169]:
path="/Users/keiikegami/Desktop/CREPE/simulator/所得税コード/sample.csv"
data=pd.read_csv(path)
#data.sort_values(by=["hh_id_use"], ascending=True)

In [170]:
rel = data["hh_relation"].unique()
mar = data["marry"].unique()
work = data["work_status"].unique()
hhid = data["hh_id_use"].unique()

#itr.product(rel, mar, work)

In [171]:
rel = ['世帯主', '世帯主の配偶者', '子', 'その他', '他の親族', '兄弟姉妹', '世帯主の配偶者の父母', '孫',
       '世帯主の父母', np.nan, '子の配偶者', '祖父母', '住み込みの雇人']

In [172]:
# そのうち重要なのは
main_relation = ['世帯主', '世帯主の配偶者', '子', '孫', '世帯主の父母']
abnormal = np.array(list(set(rel) - set(main_relation)))

In [173]:
len(hhid)

534727

In [174]:
# 真っ当な家族だけに絞る

# 家族の人数で絞る
famnum = data.groupby("hh_id_use", as_index=False).size()
normal_family_index = np.array(fam_num[fam_num <= 7].index.values)
data = data[data["hh_id_use"].isin(normal_family_index)]

In [175]:
# 家族の構成で絞る
def check_compose(s):
    if np.any(s.values in abnormal):
        return 0
    else:
        return 1
    
check = data.groupby("hh_id_use", as_index=False).agg({"hh_relation":check_compose})

/Users/keiikegami/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [176]:
normal_family_index2 = check[check["hh_relation"] == 1]["hh_id_use"].values
data = data[data["hh_id_use"].isin(normal_family_index2)]

これで全ての家系が七人以下の構成で、hh_relationは

['世帯主', '世帯主の配偶者', '子', '孫', '世帯主の父母']　しかいない

パターン列挙が丸そう

見たい控除は

employment_income 

exemption_for_dependents

f_window_deduction

old_age_deduction

pension_deduction

spouse_deduction

spouse_special_deduction

working_student_deduction


可能性があるところは全てflagを立てておくのが良さそう

家族構成からわかる奴

exemption_for_dependents

spouse_deduction

### 配偶者控除

In [177]:
def check_spouse(s):
    if "世帯主の配偶者" in s.values:
        return 1
    else:
        return 0

spouse = data.groupby("hh_id_use").agg({"hh_relation":check_spouse})

In [178]:
spouse = spouse.rename(columns={'hh_relation': 'spouse_deduction'})
data = pd.merge(data, spouse, left_on='hh_id_use', right_index=True)
data["spouse_deduction"].where(data["hh_relation"] == "世帯主", 0 , inplace=True)

## 家計のサイズが2以上だったらexemption_for_depは1にしよう

In [181]:
famnum = data.groupby("hh_id_use", as_index=False).size()
fam_num.columns = ["fam_size"]
data = pd.merge(data, fam_num, left_on='hh_id_use', right_index=True)
data["fam_size"].where(data["fam_size"] > 1, 0, inplace=True)
data["fam_size"].mask(data["fam_size"] > 1, 1, inplace=True)
data["fam_size"].where(data["hh_relation"] == "世帯主", 0 , inplace=True)

In [182]:
data = data.rename(columns={"fam_size":"exemption_for_dependents"})
data.head()

,birth_gengo,birth_year,birth_month,hh_relation,sex,age,marry,work_status,hh_id_use,spouse_deduction,exemption_for_dependents,exemption_for_dependents
0,昭和,46.0,1.0,世帯主,female,44.0,NaN,主に仕事,50,0,0,0
1,昭和,37.0,10.0,世帯主,male,52.0,配偶者あり,主に仕事,125,0,0,0
2,NaN,NaN,NaN,世帯主,male,NaN,NaN,NaN,183,0,0,0
3,昭和,43.0,7.0,世帯主,female,47.0,NaN,主に仕事,325,0,0,0
4,昭和,61.0,8.0,世帯主,female,29.0,未婚,NaN,468,0,0,0
